In [2]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from get_lat_lon import get_lat_lon_from_address

import tqdm
import requests
from bs4 import BeautifulSoup
import time
import geocoder

In [9]:
# Read recipe inputs
raw_ramen = dataiku.Dataset("raw_ramen_locations")
df = raw_ramen.get_dataframe()

In [10]:
df.head()

,store_id,store_name,address,都,市,区,番地,address_cleaned,score,ward,review_cnt,review,review_star,addresslat,addresslon,addressgranularity


In [4]:
addresses = df['address'].tolist()

In [13]:
def geocoding_lat_lon(address):
    g = geocoder.mapquest(address, key='eOUDWog4FKpjWQmPZWRCzhiKr3GW0mEr')
    latitude = g.json['raw']['latLng']['lat']
    longitude = g.json['raw']['latLng']['lng']
    return latitude, longitude

In [15]:
df['latitude'], df['longitude'] = zip(*df['address'].map(geocoding_lat_lon))

ValueError: not enough values to unpack (expected 2, got 0)

In [5]:
lat_lons = get_lat_lon_from_address(addresses)

  1%|          | 5/736 [00:14<34:53,  2.86s/it]


ValueError: Invalid address submitted. 東京都中野区中野5-56-121F

In [11]:
url = 'http://www.geocoding.jp/api/'
latlons = []
address = '東京都中野区中野5-56-121'
payload = {"v": 1.1, 'q': address}
r = requests.get(url, params=payload)
ret = BeautifulSoup(r.content,'lxml')
if ret.find('error'):
    raise ValueError(f"Invalid address submitted. {address}")
else:
    lat = ret.find('lat').string
    lon = ret.find('lng').string
    latlons.append([lat,lon])
    #time.sleep(10)

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

lat_lon_df = raw_ramen_df # For this sample code, simply copy input to output


# Write recipe outputs
lat_lon = dataiku.Dataset("lat_lon")
lat_lon.write_with_schema(lat_lon_df)